In [1]:
# import
import pandas as pd
import numpy as np

In [2]:
# load data
start = "../"
# start = "/mnt/c/Users/dalli/source/acme_senior/projectV3/"
train = pd.read_csv(start + 'data/train.csv')
test = pd.read_csv(start + 'data/test.csv')

# show the first and last times in the test set pickup_datetime
# concert the pickup_datetime column to a datetime object
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
train["dropoff_datetime"] = pd.to_datetime(train["dropoff_datetime"])
test['pickup_datetime'] = pd.to_datetime(test['pickup_datetime'])

print("Date Range:")
print(train['pickup_datetime'].min())
print(train['pickup_datetime'].max())

print("\nGrid Range:")
min_lat = min(train["pickup_latitude"].min(), train['dropoff_latitude'].min())
min_long = min(train["pickup_longitude"].min(), train['dropoff_longitude'].min())
max_lat = max(train["pickup_latitude"].max(), train['dropoff_latitude'].max())
max_long = max(train["pickup_longitude"].max(), train['dropoff_longitude'].max())
print(min_lat, min_long)
print(max_lat, max_long)

# drop extra columns
train = train.drop(['id', 'vendor_id', "store_and_fwd_flag", 'dropoff_datetime'], axis=1)
test = test.drop(['id', 'vendor_id', "store_and_fwd_flag"], axis=1)

# apply the floor("H") operation to every row in test["hour"]
# train['hour'] = train['pickup_datetime'].apply(lambda date: date.floor('H'))
train['hour'] = train['pickup_datetime'].dt.floor('H')
test['hour'] = test['pickup_datetime'].dt.floor('H')

Date Range:
2016-01-01 00:00:17
2016-06-30 23:59:39

Grid Range:
32.1811408996582 -121.93334197998048
51.88108444213867 -61.33552932739258


In [3]:
train.head()

,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,hour
0,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,455,2016-03-14 17:00:00
1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,663,2016-06-12 00:00:00
2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,2124,2016-01-19 11:00:00
3,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,429,2016-04-06 19:00:00
4,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,435,2016-03-26 13:00:00


In [4]:
# load weather data
# https://www.kaggle.com/datasets/aadimator/nyc-weather-2016-to-2022
weather = pd.read_csv(start + 'data/weather.csv')
weather = weather.dropna()

# drop all rows where the time is after 2016-07-01
weather['time'] = pd.to_datetime(weather['time'])
weather = weather[weather['time'] <= '2016-07-01']

# drop extraneous columns and show
weather = weather.drop(columns=['rain (mm)', 'cloudcover_low (%)', 'cloudcover_mid (%)', 'cloudcover_high (%)', 'windspeed_10m (km/h)', 'winddirection_10m (°)'])
print(weather['time'].min())
print(weather['time'].max())

2016-01-01 00:00:00
2016-07-01 00:00:00


In [5]:
# left join test and weather on hour == time
taxis = train.merge(weather, left_on='hour', right_on='time', how='left')
taxis = taxis.drop(columns=['time', 'hour'])
validation = test.merge(weather, left_on='hour', right_on='time', how='left')
validation = validation.drop(columns=['time', 'hour'])

In [6]:
taxis.head()

,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,temperature_2m (°C),precipitation (mm),cloudcover (%)
0,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,455,6.4,0.2,100.0
1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,663,28.8,0.0,5.0
2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,2124,-7.6,0.0,0.0
3,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,429,7.9,0.0,2.0
4,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,435,4.7,0.0,5.0


In [7]:
taxis.describe()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,temperature_2m (°C),precipitation (mm),cloudcover (%)
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02,1.029680e+01,9.552324e-02,4.417891e+01
std,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03,9.031819e+00,4.173544e-01,3.770505e+01
min,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00,-1.680000e+01,0.000000e+00,0.000000e+00
25%,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02,3.200000e+00,0.000000e+00,7.000000e+00
50%,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02,1.030000e+01,0.000000e+00,3.300000e+01
75%,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03,1.710000e+01,0.000000e+00,8.500000e+01
max,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06,3.070000e+01,7.900000e+00,1.000000e+02


In [8]:
taxis.dtypes

pickup_datetime        datetime64[ns]
passenger_count                 int64
pickup_longitude              float64
pickup_latitude               float64
dropoff_longitude             float64
dropoff_latitude              float64
trip_duration                   int64
temperature_2m (°C)           float64
precipitation (mm)            float64
cloudcover (%)                float64
dtype: object

In [9]:
# save the data as a csv
# taxis.to_csv(start + 'data/taxis.csv', index=False)
taxis['pickup_float'] = pd.to_datetime(taxis['pickup_datetime']).astype('int64') // 10**10
taxis.head()

C:\Users\dalli\AppData\Local\Temp\ipykernel_18628\2023588461.py:3: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  taxis['pickup_float'] = pd.to_datetime(taxis['pickup_datetime']).astype('int64') // 10**10


,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,temperature_2m (°C),precipitation (mm),cloudcover (%),pickup_float
0,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,455,6.4,0.2,100.0,145797629
1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,663,28.8,0.0,5.0,146569221
2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,2124,-7.6,0.0,0.0,145320332
3,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,429,7.9,0.0,2.0,145997115
4,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,435,4.7,0.0,5.0,145899905


In [10]:
taxis['pickup_float'] = (taxis['pickup_datetime'] - taxis['pickup_datetime'].min()) / (taxis['pickup_datetime'].max() - taxis['pickup_datetime'].min())
taxis.head()

,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,temperature_2m (°C),precipitation (mm),cloudcover (%),pickup_float
0,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,455,6.4,0.2,100.0,0.405086
1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,663,28.8,0.0,5.0,0.895772
2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,2124,-7.6,0.0,0.0,0.101554
3,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,429,7.9,0.0,2.0,0.531947
4,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,435,4.7,0.0,5.0,0.470127


In [11]:
# convert datetimes to ints and normalize
# taxis['pickup_float'] = pd.to_datetime(taxis['pickup_datetime']).astype('int64') // 10**9
# taxis['pickup_float'] = (taxis['pickup_datetime'] - taxis['pickup_datetime'].min()) / (taxis['pickup_datetime'].max() - taxis['pickup_datetime'].min())

# make train and label data
train_data = taxis.drop(columns=['trip_duration', 'pickup_datetime'])
train_labels = taxis['trip_duration']

taxis.head()

,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,temperature_2m (°C),precipitation (mm),cloudcover (%),pickup_float
0,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,455,6.4,0.2,100.0,0.405086
1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,663,28.8,0.0,5.0,0.895772
2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,2124,-7.6,0.0,0.0,0.101554
3,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,429,7.9,0.0,2.0,0.531947
4,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,435,4.7,0.0,5.0,0.470127


In [12]:
# make train and test splits
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# make a random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=40, 
                            max_depth=5, 
                            max_leaf_nodes=5, 
                            random_state=0, 
                            verbose=4, 
                            max_samples=0.1, 
                            n_jobs=-2)
rf.fit(train_data, train_labels)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


building tree 1 of 40building tree 2 of 40

building tree 3 of 40
building tree 4 of 40
building tree 5 of 40
building tree 6 of 40
building tree 7 of 40
building tree 8 of 40
building tree 9 of 40
building tree 10 of 40
building tree 11 of 40
building tree 12 of 40
building tree 13 of 40
building tree 14 of 40
building tree 15 of 40
building tree 16 of 40
building tree 17 of 40
building tree 18 of 40


[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:   30.0s


building tree 19 of 40
building tree 20 of 40
building tree 21 of 40
building tree 22 of 40
building tree 23 of 40
building tree 24 of 40
building tree 25 of 40
building tree 26 of 40
building tree 27 of 40
building tree 28 of 40
building tree 29 of 40
building tree 30 of 40
building tree 31 of 40
building tree 32 of 40
building tree 33 of 40
building tree 34 of 40
building tree 35 of 40
building tree 36 of 40
building tree 37 of 40
building tree 38 of 40
building tree 39 of 40
building tree 40 of 40


[Parallel(n_jobs=-2)]: Done  38 out of  40 | elapsed:  1.6min remaining:    4.9s
[Parallel(n_jobs=-2)]: Done  40 out of  40 | elapsed:  1.6min finished


TypeError: predict() takes 2 positional arguments but 3 were given

In [18]:
# perform validation
# predicted_labels = rf.predict(test_data)
# print results
# get 10% of test data and test labels
small_test_data = test_data[:int(len(test_data) * 0.1)]
small_test_labels = test_labels[:int(len(test_labels) * 0.1)]
print("Accuracy:", rf.score(small_test_data, small_test_labels))

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
